# Improved ID3 algorithm for clinical data classification

In [5]:
# Import the libraries used
import unicodecsv
from collections import defaultdict
from math import cos,sqrt,log2

In [6]:
# Read the data set into reader
diagnosis = []
f = open('breast.csv', 'rb')
reader = unicodecsv.DictReader(f)
for row in reader:
    diagnosis.append(row)
f.close()

In [7]:
# Length of read data set
print(len(diagnosis))
# A tuple in the data set 
diagnosis[0]

699


OrderedDict([('id', '1000025'),
             ('clump', '5'),
             ('size', '1'),
             ('shape', '1'),
             ('adhesion', '1'),
             ('epithelial', '2'),
             ('barenuclei', '1'),
             ('bland', '3'),
             ('normal', '1'),
             ('mitoses', '1'),
             ('class', '2')])

In [8]:
# Data Wrangling: If the tuple contains missing values remove that tuple
def check(data):
    for v in data.values():
        if v=='?':
            return False
    return True

In [9]:
# clean the data and store it in the cleanedData variable
cleanedData=[]
for diag in diagnosis:
    if check(diag):
        diag.pop('id')
        cleanedData.append(diag)
    

In [10]:
# Node in a ID3 tree
class Node:
    def __init__(self,testAttr,predictedClass):
        self.testAttr=testAttr
        self.predictedClass=predictedClass
        self.children={}
    def __str__(self):
        return self.testAttr
def printTree(n):
    print(n)
    if len(n.children)==0:
        return
    print('children are:')
    for k,v in n.children.items():
        print(k,v)
    for v in n.children.values():
        printTree(v)

In [11]:
# Import breastAttr.txt into breastAttr
# The breastAttr contains attributes in the data set and their possible values
with open('breastAttr.txt','r') as f:
    breastAttr=f.read()
s=breastAttr.split('\n')

In [12]:
# possVals is a dictionary containing attrname and its possible values
possVals={}
for line in s:
    temp=line.split(' ')
    possVals[temp[0]]=temp[1].split(',')
possVals

{'adhesion': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'barenuclei': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'bland': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'clump': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'epithelial': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'mitoses': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'normal': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'shape': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'size': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']}

In [13]:
# From breastClass get the classes present and their respective values
possClassValues=[]
with open('breastClass.txt','r') as f:
    breastClass=f.read()
classLine=breastClass.split(' ')
className = classLine[0]
possClassValues = classLine[1].split(',')


In [14]:
print(possClassValues,className)

['2', '4'] class


In [15]:
# Balance function which decides on which attribute the node should split the tree
def entropy(edata):
    temp = defaultdict(int)
    for eavpair in edata:
        temp[eavpair[className]]+=1
    en = len(edata)
    e=1
    for epv in temp.values():
        e*=(epv/en)*log2(epv/en)
    return e

In [16]:
# IID3 improvisation: balance function to reduce multivariate splits
def balance(attrLen):
    ans=cos(3.5*attrLen-1.5)/1.8
    ans*=log2(sqrt(attrLen+1))
    return abs(ans)

In [17]:
# The Improved ID3 algorithm written here as train
# The algorithm return the root of the tree
def train(trainData,attributes):
    e = entropy(trainData)
    if e==0:
        return Node('',trainData[0][className])
    if len(attributes) == 0:
        classCount = defaultdict(int)
        for ex in trainData:
            classCount[ex[className]] += 1
        maxClass= possClassValues[0]
        maxCount= classCount[maxClass]
        for k,v in classCount.items():
            if maxCount<v:
                maxCount = v
                maxClass = k
        return Node('',maxClass)
    info = {}
    gain = {}
    n = len(trainData)
    for attr in attributes:
        attrVals = possVals[attr]
        attrTrainData = {}
        attrTrainData = defaultdict(list)
        for avpair in trainData:
            attrTrainData[avpair[attr]].append(avpair)
        info[attr] = 1
        for attrVal in attrTrainData.values():
            info[attr] += (len(attrVal)/n)*entropy(attrVal)
            gain[attr] = (e-info[attr])/balance(len(attrVals))
    maxAttr = attributes[0]
    maxGain = gain[maxAttr]
    for k,v in gain.items():
        if maxGain < v:
            maxGain = v
            maxAttr = k
    n=Node(maxAttr,'')
    splitTrainData = defaultdict(list)
    for ex in trainData:
        splitTrainData[ex[maxAttr]].append(ex)
    newAttrs = attributes[:]
    newAttrs.remove(maxAttr)
    for splitAttrVal, splitData in splitTrainData.items():
        n.children[splitAttrVal]=train(splitData,newAttrs)
    return n

In [18]:
n=train(cleanedData[136:],list(possVals.keys()))

In [19]:
# Testing function to check the accuracy of the algorithm
def testing1(n,trainData):
    ab=0
    x=0
    for ex in trainData:
        p=n
        try:
            while p.testAttr!='':
                child=ex[p.testAttr]
                p=p.children[child]
            print(p.predictedClass,ex[className])
            if p.predictedClass==ex[className]:
                x+=1
        except:
            ab+=1
    return x,ab

In [ ]:
x=testing1(n,cleanedData[:136])